In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Google Drive'ı bağla (eğer zaten bağlı değilse)
drive.mount('/content/drive')

image_dir = '/content/drive/My Drive/prostatemobilenet/dataset/TMA_images'
mask_dir = '/content/drive/MyDrive/prostatemobilenet/dataset/Gleason_masks_test/Gleason_masks_test_pathologist1'

# Maskelerdeki renklerin RGB değerleri ve karşılık gelen etiketler
# Bu RGB değerlerinin, maske dosyanızdaki tam değerler olduğundan emin olun.
# Önceki görseldeki renkler göz önüne alınarak tahmin edilen değerler:
# Ancak, maskeyi bir görüntü düzenleyicide açıp RGB değerlerini kontrol etmek EN DOĞRU yol olacaktır.
# Örneğin, bir piksel seçici ile tam RGB değerlerini alabilirsiniz.
# Şimdilik, standart kırmızı, yeşil, mavi, sarı ve beyaz değerlerini kullanacağım.
# Eğer çıktıdaki görselleştirme yanlış görünürse, bu RGB değerlerini düzeltmemiz gerekecek.

COLOR_TO_LABEL = {
    (0, 255, 0): 0,    # Yeşil -> Benign
    (0, 0, 255): 1,    # Mavi -> Gleason_3
    (255, 255, 0): 2,  # Sarı -> Gleason_4
    (255, 0, 0): 3,    # Kırmızı -> Gleason_5
    (255, 255, 255): 4 # Beyaz -> Unlabelled
}

# Etiketleri tersine çevirerek de kullanışlı olabilir (görselleştirme için)
LABEL_TO_COLOR = {v: k for k, v in COLOR_TO_LABEL.items()}

# Şimdi önceki eşleştirme ve temizleme kodumuzu çalıştıralım ve ardından patch oluşturmaya geçelim.

image_files = sorted([f for f in os.listdir(image_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tif', '.tiff'))])

# Bozuk görüntü dosyalarını kontrol et
bad_images = []
good_image_filenames = []
print("Bozuk görüntü dosyaları kontrol ediliyor...")
for i, filename in enumerate(image_files):
    filepath = os.path.join(image_dir, filename)
    try:
        with Image.open(filepath) as img:
            img.verify()
            img.close()
        img_cv = cv2.imread(filepath)
        if img_cv is None or img_cv.shape[0] == 0 or img_cv.shape[1] == 0:
            raise ValueError(f"OpenCV görüntüyü okuyamadı veya boyutu sıfır: {filename}")
        good_image_filenames.append(filename)
    except Exception as e:
        print(f"Hata: {filename} dosyası okunamadı veya bozuk. Sebep: {e}")
        bad_images.append(filename)
print(f"Toplam {len(bad_images)} bozuk görüntü dosyası bulundu.")

# Sadece maskesi olan ve bozuk olmayan görüntü-maske çiftlerini alalım
image_mask_pairs = []
for img_filename in good_image_filenames: # Sadece iyi görüntüleri kontrol et
    base_name, img_ext = os.path.splitext(img_filename)
    expected_mask_filename = "mask1_" + base_name + ".png"

    if os.path.exists(os.path.join(mask_dir, expected_mask_filename)):
        image_mask_pairs.append((img_filename, expected_mask_filename))

print(f"\nMaskesi olan ve kullanılacak toplam {len(image_mask_pairs)} görüntü-maske çifti bulundu (bozuk dosyalar hariç).")

# --- Maske Dönüştürme ve Patch Oluşturma Fonksiyonları ---

def convert_mask_to_labels(mask_rgb_image, color_to_label_map):
    """
    RGB maskeyi, sayısal etiketlere sahip tek kanallı maskeye dönüştürür.
    mask_rgb_image: PIL Image nesnesi, RGB modunda.
    color_to_label_map: Sözlük, (R, G, B) tuple'larından etiket ID'lerine (int) eşleştirme.
    """
    # PIL Image'ı NumPy array'e çevir
    mask_rgb = np.array(mask_rgb_image)
    h, w, _ = mask_rgb.shape
    label_mask = np.zeros((h, w), dtype=np.uint8) # Etiketler için uint8 (0-255) yeterli

    # Her pikselin rengini kontrol edip etikete dönüştür
    # Daha hızlı bir yol: her renk için maske oluşturup bunları birleştirmek
    for color_tuple, label_id in color_to_label_map.items():
        # Bu renge sahip pikselleri bul
        # np.all() kullanarak tüm kanallarda eşleşme kontrolü
        matches = np.all(mask_rgb == np.array(color_tuple).reshape(1, 1, 3), axis=-1)
        label_mask[matches] = label_id

    return label_mask


def create_patches(image_path, mask_path, output_image_dir, output_mask_dir, patch_size, overlap_ratio, color_to_label_map, min_labeled_pixels_ratio=0.1):
    """
    Büyük görüntülerden ve maskelerden patch'ler oluşturur ve kaydeder.
    min_labeled_pixels_ratio: Bir patch'in en az bu oranda etiketli piksel içermesi gerekir.
                               (Arka plan veya "unlabelled" hariç)
    """
    img = cv2.imread(image_path)
    if img is None:
        print(f"Hata: Görüntü okunamadı: {image_path}")
        return

    # PIL ile maskeyi RGB olarak oku
    try:
        mask_pil = Image.open(mask_path).convert('RGB')
        # Maskenin orijinal boyutunu kontrol etmek iyi bir pratik
        if img.shape[0:2] != mask_pil.size[::-1]: # PIL (width, height), NumPy (height, width)
             print(f"Hata: Görüntü ve maske boyutları uyuşmuyor: Görüntü {img.shape[:2]}, Maske {mask_pil.size[::-1]}")
             return
    except Exception as e:
        print(f"Hata: Maske okunamadı veya RGB'ye çevrilemedi: {mask_path}. Sebep: {e}")
        return

    # RGB maskeyi sayısal etiket maskesine dönüştür
    label_mask = convert_mask_to_labels(mask_pil, color_to_label_map)

    h, w, _ = img.shape
    step_size = int(patch_size * (1 - overlap_ratio))
    if step_size == 0: step_size = patch_size # Tam çakışma veya çakışma yoksa

    patch_count = 0
    base_filename = os.path.splitext(os.path.basename(image_path))[0]

    # Etiketli piksel kontrolü için 'unlabelled' sınıfının etiketi
    unlabelled_id = COLOR_TO_LABEL.get((255, 255, 255), None) # Beyazı unlabelled varsayıyoruz

    for y in range(0, h - patch_size + 1, step_size):
        for x in range(0, w - patch_size + 1, step_size):
            img_patch = img[y:y+patch_size, x:x+patch_size]
            mask_patch = label_mask[y:y+patch_size, x:x+patch_size]

            # En az bir miktar "gerçek" etiketli piksel içeriyor mu kontrolü
            # Arka plan ve unlabelled (etiketsiz) pikselleri dışarıda bırak
            if unlabelled_id is not None:
                # Sadece Gleason sınıflarına ait pikselleri say
                relevant_pixels = mask_patch[mask_patch != unlabelled_id]

                # Sadece Benign, Gleason_3, Gleason_4, Gleason_5 sınıfları varsa
                # Burada 0 (Benign) de sayılmalı mı, yoksa sadece kanserli alanlar mı?
                # Genellikle Benign de bir "gerçek" doku sınıfıdır.

                # Eğer sadece kanserli alanları saymak istiyorsanız:
                # relevant_pixels = mask_patch[(mask_patch != unlabelled_id) & (mask_patch != COLOR_TO_LABEL[(0,255,0)])] # Eğer benign (yeşil) de sayılmayacaksa

                if len(relevant_pixels) < (patch_size * patch_size * min_labeled_pixels_ratio):
                    continue # Yeterli etiketli piksel yoksa bu patch'i atla
            else:
                # Eğer unlabelled sınıfı yoksa veya tanımlı değilse, tüm etiketli pikselleri say
                if np.count_nonzero(mask_patch) < (patch_size * patch_size * min_labeled_pixels_ratio):
                    continue # Yeterli etiketli piksel yoksa bu patch'i atla


            # Patch'leri kaydet
            patch_name_id = f"{base_filename}_patch_{y}_{x}" # Benzersiz kimlik
            img_patch_name = f"{patch_name_id}.png" # Görüntü patch'i PNG olabilir
            mask_patch_name = f"mask_{patch_name_id}.png" # Maske patch'i PNG olmalı (etiketleri korur)

            # Görüntü patch'i
            cv2.imwrite(os.path.join(output_image_dir, img_patch_name), img_patch)

            # Maske patch'i (tek kanallı, etiketli). PNG, etiketsiz sıkıştırma ile en iyisidir.
            cv2.imwrite(os.path.join(output_mask_dir, mask_patch_name), mask_patch)

            patch_count += 1

    print(f"'{os.path.basename(image_path)}' için {patch_count} adet uygun patch oluşturuldu.")

# --- Ana Çalıştırma Bloğu ---

# Çıktı klasörlerini hazırla
output_base_dir = '/content/drive/My Drive/prostatemobilenet/dataset_patches'
output_train_images = os.path.join(output_base_dir, 'test_images')
output_train_masks = os.path.join(output_base_dir, 'test_masks')

os.makedirs(output_train_images, exist_ok=True)
os.makedirs(output_train_masks, exist_ok=True)

# Patch boyutları ve çakışma oranı
patch_size = 256 # 256x256 patchler
overlap_ratio = 0.25 # %25 çakışma. (256 * 0.75 = 192 pikselde bir kayma)
min_labeled_pixels_ratio = 0.05 # Bir patch'in en az %5'i etiketli piksel içermelidir (unlabelled hariç)

print(f"\nPatch oluşturma işlemi başlatılıyor...")
print(f"Patch Boyutu: {patch_size}x{patch_size}")
print(f"Çakışma Oranı: %{overlap_ratio*100}")
print(f"Min. Etiketli Piksel Oranı: %{min_labeled_pixels_ratio*100} (unlabelled hariç)")

# Tüm eşleşen görüntü-maske çiftleri üzerinde döngü
for img_name, mask_name in image_mask_pairs:
    img_path = os.path.join(image_dir, img_name)
    mask_path = os.path.join(mask_dir, mask_name)
    create_patches(img_path, mask_path, output_train_images, output_train_masks, patch_size, overlap_ratio, COLOR_TO_LABEL, min_labeled_pixels_ratio)

print("\nPatch oluşturma işlemi tamamlandı.")
print(f"Oluşturulan görüntü patch'leri: {output_train_images}")
print(f"Oluşturulan maske patch'leri: {output_train_masks}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Bozuk görüntü dosyaları kontrol ediliyor...
Toplam 0 bozuk görüntü dosyası bulundu.

Maskesi olan ve kullanılacak toplam 245 görüntü-maske çifti bulundu (bozuk dosyalar hariç).

Patch oluşturma işlemi başlatılıyor...
Patch Boyutu: 256x256
Çakışma Oranı: %25.0
Min. Etiketli Piksel Oranı: %5.0 (unlabelled hariç)
'ZT80_38_A_1_1.jpg' için 178 adet uygun patch oluşturuldu.
'ZT80_38_A_1_10.jpg' için 160 adet uygun patch oluşturuldu.
'ZT80_38_A_1_11.jpg' için 89 adet uygun patch oluşturuldu.
'ZT80_38_A_1_13.jpg' için 164 adet uygun patch oluşturuldu.
'ZT80_38_A_1_14.jpg' için 188 adet uygun patch oluşturuldu.
'ZT80_38_A_1_2.jpg' için 144 adet uygun patch oluşturuldu.
'ZT80_38_A_1_4.jpg' için 184 adet uygun patch oluşturuldu.
'ZT80_38_A_1_5.jpg' için 193 adet uygun patch oluşturuldu.
'ZT80_38_A_1_7.jpg' için 110 adet uygun patch oluşturuldu.
'ZT80_38_A_1_8.jpg' için 